In [1]:
import plotly
import open3d as o3d
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [2]:
print("Testing IO for point cloud ...")
pcd = o3d.io.read_point_cloud("TestData/test_face.pcd")
points = np.array(pcd.points)
print(pcd)
print(f"shape: {points.shape}")
print(f"point clouds:\n{points}")

Testing IO for point cloud ...
geometry::PointCloud with 632 points.
shape: (632, 3)
point clouds:
[[ 0.05903847 -0.0954631  -0.30587542]
 [ 0.04894953 -0.09534968 -0.3090339 ]
 [ 0.09583344 -0.09533548 -0.3125844 ]
 ...
 [ 0.12880625 -0.00544807 -0.30917254]
 [ 0.07888613 -0.09532544 -0.30588806]
 [ 0.0699131  -0.0800878  -0.30399999]]


In [3]:
def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()

def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=50, redraw=True),
                                                                    transition=dict(duration=0),
                                                                    fromcurrent=True,
                                                                    mode='immediate'
                                                                    )]
                                                    )
                                            ]
                                    )
                                ]
                    ),
                    frames=frames
            )

    return fig

In [4]:
x, y, z = points[:600, 0], points[:600, 1], points[:600, 2]

In [5]:
pcshow(x, y, z)

In [6]:
down_sampled = pcd.uniform_down_sample(1)
ary = np.array(down_sampled.points)
print(ary.shape)
print(ary[:10])

(632, 3)
[[ 0.05903847 -0.0954631  -0.30587542]
 [ 0.04894953 -0.09534968 -0.3090339 ]
 [ 0.09583344 -0.09533548 -0.3125844 ]
 [ 0.03106478 -0.09468865 -0.31878102]
 [ 0.11443429 -0.08767143 -0.32300001]
 [ 0.07903931 -0.08550008 -0.30508888]
 [ 0.09619871 -0.08530185 -0.31256962]
 [ 0.0409947  -0.08491655 -0.31196609]
 [ 0.10088533 -0.08567737 -0.31538519]
 [ 0.05792908 -0.07916701 -0.30425   ]]


In [7]:
x, y, z = ary[:, 0], ary[:, 1], ary[:, 2]
pcshow(x, y, z)

In [8]:
# normalize

def zscore(x, axis = None):
    xmean = x.mean(axis=axis, keepdims=True)
    xstd  = np.std(x, axis=axis, keepdims=True)
    zscore = (x-xmean)/xstd
    return zscore

print(f"default points:\n{ary[:10]}")
norm_pcd = zscore(ary, axis=0)
print(f"normalized points:\n{norm_pcd[:10]}")
print(f"standard: {norm_pcd.std()}")

default points:
[[ 0.05903847 -0.0954631  -0.30587542]
 [ 0.04894953 -0.09534968 -0.3090339 ]
 [ 0.09583344 -0.09533548 -0.3125844 ]
 [ 0.03106478 -0.09468865 -0.31878102]
 [ 0.11443429 -0.08767143 -0.32300001]
 [ 0.07903931 -0.08550008 -0.30508888]
 [ 0.09619871 -0.08530185 -0.31256962]
 [ 0.0409947  -0.08491655 -0.31196609]
 [ 0.10088533 -0.08567737 -0.31538519]
 [ 0.05792908 -0.07916701 -0.30425   ]]
normalized points:
[[-1.30673683e-05 -1.83562510e+00 -1.77737432e-01]
 [-2.15073019e-01 -1.83395166e+00 -3.83357058e-01]
 [ 7.84323137e-01 -1.83374201e+00 -6.14497490e-01]
 [-5.96311437e-01 -1.82419794e+00 -1.01790239e+00]
 [ 1.18082634e+00 -1.72065651e+00 -1.29256245e+00]
 [ 4.26332886e-01 -1.68861749e+00 -1.26532847e-01]
 [ 7.92109425e-01 -1.68569265e+00 -6.13535173e-01]
 [-3.84641251e-01 -1.68000744e+00 -5.74245083e-01]
 [ 8.92011377e-01 -1.69123352e+00 -7.96831369e-01]
 [-2.36613002e-02 -1.59517102e+00 -7.19213484e-02]]
standard: 0.9999999999999997


In [9]:
x, y, z = norm_pcd[:, 0], norm_pcd[:, 1], norm_pcd[:, 2]
pcshow(x, y, z)